#### check torck is installed correctly

In [ ]:
import torch
# Check if CUDA is available
cuda_available = torch.cuda.is_available()
print(f"CUDA available: {cuda_available}")

if cuda_available:
    # Get the number of GPUs available
    num_gpus = torch.cuda.device_count()
    print(f"Number of GPUs available: {num_gpus}")

    # Get the name of each GPU
    for i in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(i)
        print(f"GPU {i}: {gpu_name}")

    # Get the current GPU memory usage
    for i in range(num_gpus):
        gpu_memory_allocated = torch.cuda.memory_allocated(i)
        gpu_memory_reserved = torch.cuda.memory_reserved(i)
        print(f"GPU {i} memory allocated: {gpu_memory_allocated / (1024 ** 3):.2f} GB")
        print(f"GPU {i} memory reserved: {gpu_memory_reserved / (1024 ** 3):.2f} GB")
else:
    print("CUDA is not available.")

#### Required Imports

In [ ]:
import sys
import os
import pickle
import logging
import numba
import matplotlib
import matplotlib.pyplot as plt
logging.getLogger('matplotlib.font_manager').disabled = True
numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

matplotlib_logger = logging.getLogger('matplotlib')
matplotlib_logger.setLevel(logging.WARNING)

# Add the src directory to sys.path
src_path = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(src_path)

from src.IndependentSteps import Pycromanager2NativeDataType, FFF2NativeDataType

from src.SequentialSteps import BIGFISH_SpotDetection, SimpleCellposeSegmentaion

from src.FinalizationSteps import Save_Outputs, Save_Images, Save_Parameters, Save_Masks, return_to_NAS, remove_local_data_but_keep_h5, \
                                    remove_all_temp, remove_temp

from src.Parameters import Parameters, Experiment, Settings, ScopeClass, DataContainer

from src.Displays import Display

from src.GUI import GUI, StepGUI

from src.Pipeline import Pipeline

#### These need to be initialized every time
These classes can only be initialized once, all future  
All future initializations will return these classes

In [ ]:
# These classes are used to isolate the source of the parameter
scope = ScopeClass() # these come from the microscope
data = DataContainer() # these hold the data as it is being generated
                        # it employ a temp file that is used to keep data out of 
                        # memory, This trys and grabs all sources of large memory
                        # memory variables and saves them
settings = Settings(name='PUT_YOUR_PIPELINENAME_HERE') # this is how you alter the 
                                                        # the settings of the pipeline,
                                                        # it is also the overflow
experiment = Experiment() # This data is specific to the experiment that you are doing.
                          # In theory this along with the scope class are the physical
                          # experiment parts need to run all image processing on that 
                          # data

#### example
Display some attribute of the settings object

In [ ]:
print(Settings().name) # this will show your name
# this is equivelent print(settings.name)

Alter some parameters

In [ ]:
experiment.initial_data_location = ['smFISH_images/Eric_smFISH_images/20220225/DUSP1_Dex_0min_20220224']
                                    # 'smFISH_images/Eric_smFISH_images/20220225/DUSP1_Dex_10min_20220224']
# this data is of the experiment class because it is the initial saving location of the
# data. once it is local in the DataContainer. This intial data location will be overwriten
# with whatever you send back, so only add what you think is useful
# you can also trim older datasets this way. It can be a list[str] or str

In [ ]:
# these properties are classic important paramters for the types of analysis this is
# built for. Primarily ICC, FISH, and live cells single molecule experiments. 
# diffusion are future used
experiment.FISHChannel = 0
experiment.nucChannel = 2
experiment.cytoChannel = 1
experiment.voxel_size_z = 500

In [ ]:
# this specifies the number of chunck should be passed through the sequential steps.
# sequential steps, mean the step is taking in a single set of p, t pairs in a specified order
# p then t or t then p
# and all results from this will be concatinated (vetically if df)
settings.num_chunks_to_run = 1

In [ ]:
# classic propeties of the microscope itself. 
# please take note that the z step is in the experiment object
scope.spot_yx = 130
scope.spot_z = 360
scope.voxel_size_yx = 100


####  You can get all the parameter objects

In [ ]:
Parameters._instances 
# returns a list of all of the scope, data, settings, experiment objects 

#### You can also get a dictionary of them all

In [ ]:
# you can get all of the parameters that you have initialized by calling get_parameters
Parameters.get_parameters()

In [ ]:
# You can check that all the manditory parameters are set by calling validate
Parameters.validate()

# this will give you two type of response, one if its a warning like this one below
# this is just a warning because you may not have a cytoChannel,
# but it is not manditory so the pipeline will still run
# there are also errors that will stop the pipeline from running

#### Next lets import some data from the NAS

In [ ]:
# If you want this to load in the previous made masks
settings.load_in_mask = True
FFF2NativeDataType().run() # this will run the step

# current assumptions about your databridges:
# it exist in a folder, this can have many things in it.
# this function looks at the folder, and loads it. It 
# will mostly populate the datacontainer, and possibly parameters.

# if a import is large consider make sure it ends up 
# in the data container
# all this data should be converted to h5 format for future ease of use,
# and for saving of results, original will remain unaltered. Aside from new h5 
# file inside

# NativeDataType().run() should be used if already meet this criteria 

In [ ]:
# clear up unneeded space on the disk
remove_local_data_but_keep_h5().run()

#### Lets look at what we got 

In [ ]:
Display().displayImage_maxProject(0, 0, 0) # Display is a class of common plots, it interacts with the paramters to display data
                                            # specifically for the user, not really used further in the pipeline.

# future plots created by steps should be able to be turned off by display_plots in Settings

#### Lets try tuning some steps

In [ ]:
# start with segmentation. ( this is a sequential step)
# these steps are built with ther own logic in mind so read the doc string to see all the options
# this class assume that if you are running it you want to overide the previous masks
# there are also several other options that you can set by adding them to the settings object
# if you want to alter any propety of the step, make it an input to the step, then pass what you want it to be into 
# settings and your good to go. Save any outputs of the step as a dictionary, and it will be save properly. 
settings.cellpose_min_size = 500
settings.cellpose_diameter = [180, 90] # most of these options can be done for individually cyto and nuc segmentation, 
                                        # and a list can be or a single float can be passed for both
                                        # always in the order cyto, nuc
settings.cellpose_pretrained_model = ["GAPDH_cyto", 'DAPI_nuclei'] 

SimpleCellposeSegmentaion().run(p=None, t=None) # You can select 

# this code will overwrite the previous masks and a warning will be given

In [ ]:
# (This is also a sequential step, we will explain there exact difference later)
# We already specified the parameters we wanted earlier, and the results of the segmentation were added to the data container.
# now were going to do spot detection using the data from above

# examples of settings to change for big fish are:
# scope.spot_yx = 130
# scope.spot_z = 360
# ...
BIGFISH_SpotDetection().run()
